In [8]:
import matplotlib.pyplot as plt

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm


<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [12]:
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns

In [13]:
import pymc3 as pm
import theano.tensor as tt
import scipy
from scipy import optimize

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [14]:
x1 = np.repeat(1, 12)
x2 = np.array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1])
x3 = np.array([23 ,22 ,22 ,25 ,27 ,20 ,31 ,23 ,27 ,28 ,22 ,24])
x4 = x2 * x3
y = np.array([-0.87, -10.74, -3.27, -1.97, 7.50, -7.25, 17.05, 4.96, 10.40, 11.05, 0.26, 2.51])
X = np.transpose(np.array([x1, x2, x3, x4]))
dim = np.shape(X)
p = dim[1]
n = dim[0]
X1 = np.transpose(np.array([x2, x3, x4]))

olsresult = sm.OLS(y, X1).fit()
beta_ols = olsresult.params
sigma2head_ols= sum(olsresult.resid ** 2) / (n-p)

In [15]:
a = [1,1,1,1]
Sigma2_0_inv = np.diag(a)* 0.0001
beta_0 = [0,0,0,0]
nu_0 = 1
sig2_0 = sigma2head_ols

In [17]:
model_string = '''
model {
 # Likelihood
for i in range(1+n) {
y[i] ~ np.random.normal(X[i][0] * beta[0] + X[i][1] * beta[1] + X[i][2] * beta[2] + X[i][3] * beta[3], sigmainv)
}
    
# Priors
sigmainv~ np.random.gamma(nu_0/2, sig2_0*nu_0/2)
beta~ np.random.multivariate_normal(c(0,0,0,0), Sigma2.0.inv)
sigma = inv(sigmainv)
} '''


In [18]:
data = {"X": X, "nu_0": nu_0, "sig2_0": sig2_0 , "Sigma2_0_inv": Sigma2_0_inv, "y": y, "n": n}
inits = {"beta" : beta_ols, "sigmainv" : 1/(sigma2head_ols)}

In [19]:
model = pyjags.Model(model_string, data = dict(data), init = dict(inits), chains = 2)

NameError: name 'pyjags' is not defined

In [20]:
update(model, 1000)

NameError: name 'update' is not defined

In [21]:
params = ['beta','sigma']

In [22]:
samples = model.sample(10000, vars=['beta','sigma'])

NameError: name 'model' is not defined

In [ ]:
def summary(samples, varname, p=95):
values = samples[varname]
ci = np.percentile(values, [100-p, p])
print('{:<6} mean = {:>5.1f}, {}% credible interval [{:>4.1f} {:>4.1f}]'.format(
      varname, np.mean(values), p, *ci))

for varname in ['beta','sigma']: summary(samples, varname)
    
plt.plot(samples)
plt.show()

In [9]:
pd.DataFrame.as_matrix(X[[1]][,1:4])

SyntaxError: invalid syntax (4189598543.py, line 1)

array([ 1,  0, 22,  0])